# **Function calling tool**

In [2]:
from llama_index.core.tools import FunctionTool

def get_weather(location: str) ->str:
    """
    Useful for getting the weather for a given location.
    """
    print(f"Getting weather for {location}")
    return f"The weather in {location} is sunny with low temperature."

tool = FunctionTool.from_defaults(get_weather,
name= "my_weather_tool",
description="Useful for getting weather for a given location")

tool.call("New York")

Getting weather for New York


ToolOutput(content='The weather in New York is sunny with low temperature.', tool_name='my_weather_tool', raw_input={'args': ('New York',), 'kwargs': {}}, raw_output='The weather in New York is sunny with low temperature.', is_error=False)

* The name and description default to the name and docstring from the provided function


* When using an agent or LLM with function calling, the tool selected (and the arguments written for that tool) rely strongly on the tool name and description of the purpose and arguments of the tool.

# **Query Engine**

In [5]:
import chromadb
from llama_index.core import VectorStoreIndex
from llama_index.core.tools import QueryEngineTool
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.embeddings.huggingface_api import HuggingFaceInferenceAPIEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore

embed_model = HuggingFaceInferenceAPIEmbedding(model_name="BAAI/bge-small-en-v1.5")

db = chromadb.PersistentClient(path="./alfred_chroma_db")
chroma_collection = db.get_or_create_collection("alfred")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

index = VectorStoreIndex.from_vector_store(vector_store, embed_model=embed_model)

llm = HuggingFaceInferenceAPI(model_name="Qwen/Qwen2.5-Coder-32B-Instruct")
query_engine = index.as_query_engine(llm=llm)

tool = QueryEngineTool.from_defaults(query_engine,name='answer based on persona',description="alfred_chroma_db documents to answer questions based on persona")


# **Tool specs**

In [7]:
from llama_index.tools.google import GmailToolSpec

tool_spec = GmailToolSpec()
tool_spec_list = tool_spec.to_tool_list()
[(tool.metadata,tool.metadata.description) for tool in tool_spec_list]


[(ToolMetadata(description="load_data() -> List[llama_index.core.schema.Document]\nLoad emails from the user's account.", name='load_data', fn_schema=<class 'llama_index.core.tools.utils.load_data'>, return_direct=False),
  "load_data() -> List[llama_index.core.schema.Document]\nLoad emails from the user's account."),
 (ToolMetadata(description="search_messages(query: str, max_results: Optional[int] = None)\nSearches email messages given a query string and the maximum number\n        of results requested by the user\n           Returns: List of relevant message objects up to the maximum number of results.\n\n        Args:\n            query[str]: The user's query\n            max_results (Optional[int]): The maximum number of search results\n            to return.\n        ", name='search_messages', fn_schema=<class 'llama_index.core.tools.utils.search_messages'>, return_direct=False),
  "search_messages(query: str, max_results: Optional[int] = None)\nSearches email messages given a qu

# **Utility tools**

1. OnDemandToolLoader - turns Llamaindex data loader into a tool that an agent can use.load data,index and query ... on demand
2. Load and search spec tool - loading tool and search tool .. loading index it search retrives